# Connection

> Fill in a module description here

In [8]:
#| default_exp connection

In [9]:
#| export
from __future__ import annotations
from typing import Optional, Dict, Any
from snowflake.snowpark import Session
from snowflake.snowpark.exceptions import SnowparkSessionException
from snowflake.snowpark.context import get_active_session
import os
from dataclasses import dataclass
from pathlib import Path
import yaml
from tenacity import retry, stop_after_attempt, wait_exponential

# Import our new modules
from snowflake_feature_store.exceptions import ConnectionError, ConfigurationError
from snowflake_feature_store.logging import logger
from snowflake_feature_store.config import BaseModel, Field

In [ ]:
#| export
class ConnectionConfig(BaseModel):
    """Configuration for Snowflake connection"""
    user: str
    password: str
    account: str
    role: str = Field("DATA_SCIENTIST", description="Snowflake role")
    warehouse: str = Field("DS_WH_XS", description="Default warehouse")
    database: str = Field("DATASCIENCE", description="Default database")
    schema: str = Field("FEATURE_STORE_DEMO", description="Default schema")
    
    @classmethod
    def from_env(cls) -> ConnectionConfig:
        """Create connection config from environment variables"""
        try:
            return cls(
                user=os.environ["SNOWFLAKE_USER"],
                password=os.environ["SNOWFLAKE_PASSWORD"],
                account=os.environ["SNOWFLAKE_ACCOUNT"],
                role=os.getenv("SNOWFLAKE_ROLE", "DATA_SCIENTIST"),
                # warehouse=os.getenv("SNOWFLAKE_WAREHOUSE", "DS_WH_XS"),
                # database=os.getenv("SNOWFLAKE_DATABASE", "DATASCIENCE"),
                # schema=os.getenv("SNOWFLAKE_SCHEMA", "FEATURE_STORE_DEMO")
            )
        except KeyError as e:
            raise ConfigurationError(f"Missing required environment variable: {e}")
    
    @classmethod
    def from_yaml(cls, path: str|Path) -> ConnectionConfig:
        """Create connection config from YAML file"""
        try:
            with open(path) as f:
                config = yaml.safe_load(f)
            return cls(**config)
        except Exception as e:
            raise ConfigurationError(f"Error loading config from {path}: {str(e)}")


In [ ]:

#| export
class SnowflakeConnection:
    """Manages Snowflake connection and configuration"""
    def __init__(self, 
                 session: Session,
                 warehouse: Optional[str] = None,
                 database: Optional[str] = None,
                 schema: Optional[str] = None):
        """Initialize Snowflake connection
        
        Args:
            session: Active Snowflake session
            warehouse: Override default warehouse
            database: Override default database
            schema: Override default schema
        """
        self.session = session
        self.warehouse = warehouse or session.get_current_warehouse()
        self.database = database or session.get_current_database()
        self.schema = schema or session.get_current_schema()
        logger.info(f"Initialized connection to {self.database}.{self.schema}")
        
    @classmethod
    def from_config(cls, config: ConnectionConfig) -> SnowflakeConnection:
        """Create connection from config object"""
        try:
            session = Session.builder.configs(config.dict()).create()
            return cls(session)
        except Exception as e:
            raise ConnectionError(f"Failed to create session: {str(e)}")
    
    @classmethod
    def from_env(cls) -> SnowflakeConnection:
        """Create connection from environment variables"""
        return cls.from_config(ConnectionConfig.from_env())
    
    @classmethod
    def from_yaml(cls, path: str|Path) -> SnowflakeConnection:
        """Create connection from YAML config file"""
        return cls.from_config(ConnectionConfig.from_yaml(path))
    
    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential(multiplier=1, min=4, max=10),
        retry_error_callback=lambda retry_state: logger.error(
            f"Failed after {retry_state.attempt_number} attempts: {retry_state.outcome.exception()}"
        )
    )
    def execute_query(self, query: str) -> Any:
        """Execute query with retry logic"""
        try:
            return self.session.sql(query).collect()
        except Exception as e:
            raise ConnectionError(f"Query execution failed: {str(e)}")
    
    def test_connection(self) -> bool:
        """Test if connection is working"""
        try:
            self.execute_query('SELECT 1')
            logger.info("Connection test successful")
            return True
        except Exception as e:
            logger.error(f"Connection test failed: {str(e)}")
            return False
            
    def close(self):
        """Close the Snowflake session"""
        if self.session:
            try:
                self.session.close()
                logger.info("Connection closed successfully")
            except Exception as e:
                logger.error(f"Error closing connection: {str(e)}")


In [ ]:
#| export
def get_connection() -> SnowflakeConnection:
    """Get Snowflake connection from active session or environment
    
    First tries to get active session, falls back to environment variables
    
    Returns:
        SnowflakeConnection object
    
    Raises:
        ConnectionError: If connection cannot be established
    """
    try:
        # Try to get active session (e.g., in Snowflake worksheet)
        session = get_active_session()
        logger.info("Using active Snowflake session")
        return SnowflakeConnection(session)
    except SnowparkSessionException:
        # Fall back to environment variables
        logger.info("No active session found, creating new connection from environment")
        return SnowflakeConnection.from_env()


In [ ]:
#| hide
def test_connection_config():
    "Test connection configuration"
    config = ConnectionConfig(
        user='test_user',
        password='test_pass',
        account='test_account'
    )
    
    # Test defaults
    assert config.role == 'DATA_SCIENTIST'
    assert config.warehouse == 'DS_WH_XS'
    
    # Test dictionary conversion
    d = config.to_dict()
    assert d['user'] == 'test_user'
    assert d['role'] == 'DATA_SCIENTIST'

def test_yaml_config(tmp_path):
    "Test YAML configuration loading"
    config_path = tmp_path / 'config.yaml'
    config_data = """
    user: test_user
    password: test_pass
    account: test_account
    role: TEST_ROLE
    """
    config_path.write_text(config_data)
    
    config = ConnectionConfig.from_yaml(config_path)
    assert config.user == 'test_user'
    assert config.role == 'TEST_ROLE'

# Run tests
test_connection_config()
test_yaml_config(Path('/tmp'))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| eval: false
# Example usage
from snowflake_feature_store.connection import get_connection, ConnectionConfig

# Method 1: Get connection automatically
conn = get_connection()

# Method 2: From environment variables
conn = SnowflakeConnection.from_env()

# Method 3: From YAML config create yaml file
# config = ConnectionConfig.from_yaml('config.yaml')
# conn = SnowflakeConnection.from_config(config)

# Test connection
if conn.test_connection():
    print("Connected successfully!")
    
# Use connection
df = conn.session.table('MY_TABLE')

# Clean up
conn.close()


Connected successfully!
